# Benchmarks

## Initialize

In [33]:
import os
import math
import pathlib
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.feather as feather
from tqdm.auto import tqdm
from IPython.display import clear_output

import warnings
from lifelines.utils import CensoringType
from lifelines.utils import concordance_index

In [34]:
import ray
ray.shutdown()

In [35]:
node = !hostname
if "sc" in node[0]:
    base_path = "/sc-projects/sc-proj-ukb-cvd"
else: 
    base_path = "/data/analysis/ag-reils/ag-reils-shared/cardioRS"
print(base_path)

project_label = "22_retina_phewas"
project_path = f"{base_path}/results/projects/{project_label}"
figure_path = f"{project_path}/figures"
output_path = f"{project_path}/data"

pathlib.Path(figure_path).mkdir(parents=True, exist_ok=True)
pathlib.Path(output_path).mkdir(parents=True, exist_ok=True)

##### BEGIN ADAPT #####
# second best model
# wandb_name = 'aug++_convnext_s_mlp'
# wandb_id = '8ngm6apd'
# best model
wandb_name = 'aug++_convnext_s_mlp+'
wandb_id = '3p3smraz'
partitions = [0] # [i for i in range(22)]
##### END   ADAPT #####

experiment = wandb_id
experiment_path = f"{output_path}/{experiment}"
pathlib.Path(experiment_path).mkdir(parents=True, exist_ok=True)

/sc-projects/sc-proj-ukb-cvd


In [36]:
import ray
ray.init(num_cpus=24, include_dashboard=False)#, dashboard_port=24762, dashboard_host="0.0.0.0", include_dashboard=True)#, webui_url="0.0.0.0"))

{'node_ip_address': '10.32.105.6',
 'raylet_ip_address': '10.32.105.6',
 'redis_address': '10.32.105.6:35387',
 'object_store_address': '/tmp/ray/session_2022-05-25_11-17-33_762916_4151524/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-05-25_11-17-33_762916_4151524/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2022-05-25_11-17-33_762916_4151524',
 'metrics_export_port': 59292,
 'node_id': '6b7497751fb71b9a6a64897c5beb02f9f6deffb805531ae0ccd54918'}

In [37]:
import pandas as pd
endpoints = sorted([l.replace('_prevalent', '') for l in list(pd.read_csv('/sc-projects/sc-proj-ukb-cvd/results/projects/22_medical_records/data/220413/frequent_endpoints.csv').endpoint.values)])

In [38]:
in_path = pathlib.Path(f"{experiment_path}/loghs")
in_path.mkdir(parents=True, exist_ok=True)

out_path = f"{experiment_path}/coxph/input"
pathlib.Path(out_path).mkdir(parents=True, exist_ok=True)

In [39]:
models = [f.name for f in in_path.iterdir() if f.is_dir() and "ipynb_checkpoints" not in str(f)]

In [40]:
from sklearn.preprocessing import StandardScaler
import pickle
import zstandard

def read_data(fp_in, split):
    temp = pd.read_feather(f"{fp_in}/{split}.feather").set_index("eid")
    return temp   
    
def save_pickle(data, data_path):
    with open(data_path, "wb") as fh:
        cctx = zstandard.ZstdCompressor()
        with cctx.stream_writer(fh) as compressor:
            compressor.write(pickle.dumps(data, protocol=pickle.HIGHEST_PROTOCOL))
    
def read_predictions(model, partition, split):

    fp_in = f"{in_path}/{model}/{partition}"
    
    if pathlib.Path(fp_in).is_dir(): 
        temp = read_data(fp_in, split)
        return temp
    else:
        print(fp_in)
        raise NotImplementedError()

In [41]:
models

['ImageTraining_[]_ConvNeXt_MLPHead']

In [42]:
for partition in partitions: # test: in [0, 10, 21]
    for split in ["train", "valid", "test"]: # "test_left", 'test_right'
        temp = read_predictions(models[0], partition, split)
        print(partition, split, (temp.isna().sum() > 0).sum())

0 train 0
0 valid 0
0 test 0
